In [2]:
import numpy as np
import pandas as pd
import talib as ta
import yfinance as yf

In [9]:
df = yf.download("AAPL", start="2021-07-10", end="2022-07-06", interval="1h", progress=False)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-07-12 09:30:00,146.210007,146.320007,144.000000,144.615005,144.615005,28048808
2021-07-12 10:30:00,144.619995,145.250000,144.429993,144.750000,144.750000,11029806
2021-07-12 11:30:00,144.759995,145.074997,144.574997,144.835007,144.835007,5621634
2021-07-12 12:30:00,144.830002,144.869995,144.500000,144.576996,144.576996,4380059
2021-07-12 13:30:00,144.570801,144.679993,144.160004,144.660004,144.660004,6374285


In [10]:
df = df[["Open", "High", "Low", "Close", "Volume"]]
df.dropna(inplace=True)

df.head()

C:\Users\anish\AppData\Local\Temp\ipykernel_11040\1188362657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,Open,High,Low,Close,Volume
Datetime,,,,,
2021-07-12 09:30:00,146.210007,146.320007,144.000000,144.615005,28048808
2021-07-12 10:30:00,144.619995,145.250000,144.429993,144.750000,11029806
2021-07-12 11:30:00,144.759995,145.074997,144.574997,144.835007,5621634
2021-07-12 12:30:00,144.830002,144.869995,144.500000,144.576996,4380059
2021-07-12 13:30:00,144.570801,144.679993,144.160004,144.660004,6374285


In [11]:
from backtesting import Backtest
from backtesting.lib import crossover, TrailingStrategy

class BbandCross(TrailingStrategy):
    n1 = 20
    n2 = 2
    n3 = 2
    

    def init(self):
        super().init()
        self.set_trailing_sl(2.5)

        high = self.data.High
        low = self.data.Low                
        close = self.data.Close
        self.upper, self.middle, self.lower= self.I(ta.BBANDS, close, self.n1, self.n2, self.n3)
        self.atr = self.I(ta.ATR, high, low, close, 14)

    def next(self):
        super().next()

        if crossover(self.data.Close, self.upper):
            self.sl = self.data.Close[-1] - 2.5*self.atr[-1]
            self.buy(size=0.75, sl=self.sl)

        elif crossover(self.lower, self.data.Close):
            self.sl = self.data.Close[-1] + 2.5*self.atr[-1]
            self.sell(size=0.75, sl=self.sl)

In [ ]:
bt = Backtest(df, BbandCross,
            cash=100000, trade_on_close=True, exclusive_orders=True, hedging=True)

stats = bt.optimize(n1=range(10, 50, 2), n2=range(2, 10, 1), n3=range(2, 10, 1), maximize="Return [%]", constraint=lambda p: p.n1 > p.n2 and p.n2==p.n3)

stats = bt.run()

In [12]:
print("OC-Range: ", (df["Close"].iloc[-1]-df["Close"].iloc[0])/df["Close"].iloc[0])
print("HL-Range: ", (np.max(df["Close"])-np.min(df["Close"]))/np.min(df["Close"]))

OC-Range:  -0.020779343558193525
HL-Range:  0.4064849237019351


In [14]:
stats

Start                     2021-07-12 09:30:00
End                       2022-07-05 15:30:00
Duration                    358 days 06:00:00
Exposure Time [%]                   68.032314
Equity Final [$]                132986.496164
Equity Peak [$]                 134277.107166
Return [%]                          32.986496
Buy & Hold Return [%]               -2.077934
Return (Ann.) [%]                   33.599379
Volatility (Ann.) [%]               21.451053
Sharpe Ratio                         1.566328
Sortino Ratio                        3.652074
Calmar Ratio                         4.169179
Max. Drawdown [%]                   -8.058992
Avg. Drawdown [%]                   -1.493836
Max. Drawdown Duration      132 days 22:00:00
Avg. Drawdown Duration        8 days 12:00:00
# Trades                                  114
Win Rate [%]                        48.245614
Best Trade [%]                       7.411483
Worst Trade [%]                     -4.046801
Avg. Trade [%]                    

In [15]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-513,33,35,146.150101,142.399994,1923.804794,0.025659,2021-07-16 14:30:00,2021-07-19 09:30:00,2 days 19:00:00
1,-536,35,42,142.399994,144.289156,-1012.590662,-0.013267,2021-07-19 09:30:00,2021-07-20 09:30:00,1 days 00:00:00
2,-513,77,105,147.380005,146.739259,328.702464,0.004348,2021-07-27 09:30:00,2021-08-02 09:30:00,6 days 00:00:00
3,516,114,134,147.070099,146.240005,-428.328186,-0.005644,2021-08-03 11:30:00,2021-08-06 10:30:00,2 days 23:00:00
4,-517,134,136,146.240005,145.895004,178.365631,0.002359,2021-08-06 10:30:00,2021-08-06 12:30:00,0 days 02:00:00
...,...,...,...,...,...,...,...,...,...,...
109,707,1678,1684,137.800003,140.884598,2180.808472,0.022385,2022-06-23 10:30:00,2022-06-24 09:30:00,0 days 23:00:00
110,703,1684,1700,140.884598,139.092775,-1259.651171,-0.012718,2022-06-24 09:30:00,2022-06-28 11:30:00,4 days 02:00:00
111,-705,1700,1712,139.210007,134.494995,3324.083176,0.033870,2022-06-28 11:30:00,2022-06-30 09:30:00,1 days 22:00:00
112,-748,1712,1724,134.494995,138.391006,-2914.216241,-0.028968,2022-06-30 09:30:00,2022-07-01 14:30:00,1 days 05:00:00
